# Time series analysis of hyp3 InSAR products by MintPy

This notebook shows how to do time-series analysis with HyP3 InSAR product by MintPy. We assume you have already got the hyp3 InSAR products somewhere. This steps for the analysis are: clip the hyp3 INSAR product, define the config.txt file, run the time series analysis, and display the results. The sample hyp3 INSAR data are at https://jzhu-hyp3-dev.s3.us-west-2.amazonaws.com/hyp3-mintpy-example/2018_kilauea.zip. Asfar as how to produce the hyp3 INSAR product, we provide the detail steps in the tutorial(https://github.com/ASFHyP3/hyp3-docs/tree/develop/docs). 


## 0. Initial setup of the notebook

To run this notebook, you'll need a conda environment with the required dependencies. You can set up a new environment (recommended) and run the jupyter server like:

conda create -n hyp3-mintpy python=3.8 asf_search hyp3_sdk "mintpy>=1.3.2" pandas jupyter ipympl

To make you conda env accessible in the jupyter notebook, you need to do:

conda activate hyp3-mintpy
conda install -c conda-forge tensorflow
conda install -c anaconda ipykernel
python -m ipykernel install --user --name=hyp3-mintpy

To run your notebook, just:

conda activate hyp3-mintpy
jupyter notebook smallbaselineApp_hyp3_new.ipynb


### Import modules and set environment variables

In [ ]:
import os
from pathlib import Path
import glob
import zipfile
from dateutil.parser import parse as parse_date
from osgeo import gdal
import numpy as np
from mintpy import view, tsview

os.environ['WEATHER_DIR'] = '/media/jzhu4/data/mintpy_data/weather_data'

# utils function

def get_intersect_rectangle_geotiffs(filelist):
    '''
    :param data_dir: data directory storing the hyp3 products.
    :process get the smallest overlap retangular area to clip the geotiff files.
    :return:
    '''
    corners = [gdal.Info(str(dem), format='json')['cornerCoordinates'] for dem in filelist]

    ulx = max(corner['upperLeft'][0] for corner in corners)
    uly = min(corner['upperLeft'][1] for corner in corners)
    lrx = min(corner['lowerRight'][0] for corner in corners)
    lry = max(corner['lowerRight'][1] for corner in corners)
    return [ulx, uly, lrx, lry]

def prepare_hyp3_product(data_dir):
    filelist = glob.glob(f"{data_dir}/*/*_dem.tif")
    insect_box = get_intersect_rectangle_geotiffs(filelist)
    #files_for_mintpy = ['_water_mask.tif', '_corr.tif', '_unw_phase.tif', '_dem.tif', '_lv_theta.tif', '_lv_phi.tif']
    files_for_mintpy = ['_water_mask.tif', '_corr.tif', '_unw_phase.tif', '_dem.tif', '_lv_theta.tif']
    list_product_dirs = [f.path for f in os.scandir(data_dir) if f.is_dir()]

    for product_dir in list_product_dirs:
        for file_suffix in files_for_mintpy:
            product_dir = Path(product_dir)
            src_file = product_dir / f'{product_dir.name}{file_suffix}'
            dst_file = product_dir / f'{src_file.stem}_clipped{src_file.suffix}'
            gdal.Translate(destName=str(dst_file), srcDS=str(src_file), projWin=insect_box)

def unzip_files(zip_file, data_dir):
    if os.path.isfile(zip_file):
        with zipfile.ZipFile(zip_file, 'r') as fzip:
            fzip.extractall(data_dir)


### Define the parameters and create directories

In [ ]:
project_name = 'Ridgecrest'

project_home = '/media/jzhu4/data/hyp3-mintpy'

work_dir = Path(project_home) / project_name

data_dir = work_dir / 'data'

if not os.path.isdir(work_dir):
    os.makedirs(work_dir)
    print('Create directory: {}'.format(work_dir))
    
if not os.path.isdir(data_dir):
    os.makedirs(data_dir)
    print('Create directory: {}'.format(data_dir))
    
os.chdir(work_dir)
print('Go to work directory: {}'.format(work_dir))

## 1. Run MintPy routine workflow `smallbaselineApp.py`

### 1.1 Load the hyp3 InSAR data

The example dataset is from 2018 Kīlauea volcano. The dataset can be obtained through either downloading from the stagged server or producing with hyp3-sdk. Here we provide the sample dataset at https://jzhu-hyp3-dev.s3.us-west-2.amazonaws.com/hyp3-mintpy. 

In [ ]:
file = 'Ridgecrest.zip'

file_url = f'https://jzhu-hyp3-dev.s3.us-west-2.amazonaws.com/hyp3-mintpy-example/{file}'


In [ ]:
!wget {file_url} -P {data_dir}

In [ ]:
print(f'downloaded file is {data_dir}/{file}')

In [ ]:
unzip_files(f'{data_dir}/{file}', data_dir)

### 1.2 Cut geotiff files for mintpy analysis

In [ ]:
prepare_hyp3_product(data_dir)

### 1.3 Prepare the template file

In [ ]:
mintpy_config = work_dir / 'mintpy_config.txt'
mintpy_config.write_text(
f"""
mintpy.load.processor        = hyp3
##---------interferogram datasets:
mintpy.load.unwFile          = {data_dir}/*/*_unw_phase_clipped.tif
mintpy.load.corFile          = {data_dir}/*/*_corr_clipped.tif
##---------geometry datasets:
mintpy.load.demFile          = {data_dir}/*/*_dem_clipped.tif
mintpy.load.incAngleFile     = {data_dir}/*/*_lv_theta_clipped.tif
#mintpy.load.azAngleFile      = {data_dir}/*/*_lv_phi_clipped.tif
mintpy.load.waterMaskFile    = {data_dir}/*/*_water_mask_clipped.tif
""")

### 1.4 Run Time-series Analysis application

In [ ]:
! smallbaselineApp.py --work-dir {work_dir}  {mintpy_config}

## 2. Display the analysis results

There are a few scripts used to display the analysis results. There are in the MINTPY_HOME/mintpy. Here we show two majoy disaply scripts.

In [ ]:
%matplotlib widget
from mintpy import view, tsview

In [ ]:
view.main([f'{work_dir}/velocity.h5'])

In [ ]:
tsview.main([f'{work_dir}/timeseries.h5'])